## Ibis and DuckDB geospatial demo

```bash
pip install 'ibis-framework[duckdb,geospatial]'
```

In [ ]:
import ibis
from ibis import _

con = ibis.duckdb.connect(extensions=['spatial'])

In [ ]:
# look into type infrastructure
url = (
    "s3://overturemaps-us-west-2/release/2025-01-22.0/theme=base/type=infrastructure/*"
)
t = con.read_parquet(url, table_name="infra-data")
t

In [ ]:
expr = t.filter(
    _.bbox.xmin >  4.728756,
    _.bbox.ymin > 52.303596,
    _.bbox.xmax < 5.046438,
    _.bbox.ymax < 52.431064,
    ).select(["subtype", "class", "geometry", "names"])
expr

In [ ]:
ibis.options.interactive = True
ibis.options.repr.interactive.max_rows = 15

In [ ]:
ams_infra = con.read_parquet("infra-amsterdam.geoparquet", table_name = "ams_infra")
ams_infra.head()

In [ ]:
# Rename column for convenience
ams_infra = ams_infra.rename(
                infra_class="class"
            ).select(["subtype", "infra_class", "geometry"])

In [ ]:
ams_infra[["subtype"]].distinct()

In [ ]:
tourist_useful = ams_infra.filter(
    _.subtype.isin(["pedestrian", "water"]))

In [ ]:
ibis.to_sql(tourist_useful)

In [ ]:
tourist_useful[["infra_class"]].distinct()

In [ ]:
tourist_useful = tourist_useful.filter(
        _.infra_class.isin(
            [
                "drinking_water",
                "toilets",
                "bench",
                "viewpoint",
                "vending_machine",
                "atm",
            ]
        )
    )


In [ ]:
import shapely as shp

In [ ]:
# we keep it always xy ie lon,lat compat with geopandas
lon, lat = 4.881, 52.359 # van Gogh museum

In [ ]:
vg_coords = ibis.literal(shp.Point(lon, lat), type='geometry')
vg_coords

In [ ]:
#st_transform
vg_coords_m = vg_coords.convert("EPSG:4326", "EPSG:3857")
vg_coords_m

In [ ]:
tourist_useful.filter(
    tourist_useful.geometry.convert(
        "EPSG:4326", "EPSG:3857").d_within(vg_coords_m, distance=1000),
)

In [ ]:
near_vanGogh_no_benches = tourist_useful.filter(
    _.geometry.convert(
        "EPSG:4326", "EPSG:3857").d_within(vg_coords_m, distance=1000),
    _.infra_class.notin(["bench"])
)
near_vanGogh_no_benches

In [ ]:
ibis.to_sql(near_vanGogh_no_benches)

In [ ]:
near_vanGogh = near_vanGogh_no_benches.mutate( 
    dist_vanGogh = _.geometry.convert(
        "EPSG:4326", "EPSG:3857").distance(vg_coords_m))

In [ ]:
near_vanGogh.order_by("dist_vanGogh")

In [ ]:
ams_buildings =  con.read_parquet("buildings-amsterdam.geoparquet", table_name="ams_buildings")
ams_buildings.head()

In [ ]:
build_near_vanGogh = ams_buildings.filter(_.geometry.convert(
                    "EPSG:4326", "EPSG:3857").intersects(
                    vg_coords_m.buffer(100))).select(_.subtype,
                                                     _.geometry
                                                    )
build_near_vanGogh

In [ ]:
import lonboard

In [ ]:
from lonboard import Map, ScatterplotLayer, PolygonLayer

In [ ]:
from lonboard.colormap import apply_categorical_cmap 

In [ ]:
# Use color blind friendly colormap

cmap = { "drinking_water": [86, 180, 233], # light blue
         "toilets": [230, 159, 0], # yellowish
         "viewpoint":[0, 0, 0], # black
         "vending_machine": [0, 114, 178], # blue 
         "atm":[0, 158, 115],} # green-ish

In [ ]:
colors = apply_categorical_cmap(near_vanGogh.infra_class.to_pyarrow(),
                              cmap=cmap)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
points_interest = ScatterplotLayer(
    table = near_vanGogh,
    get_fill_color=colors,
    get_radius = 15
)

buildings = PolygonLayer(
    table = build_near_vanGogh, 
    get_fill_color = [204, 121, 167],
    opacity=0.3
)

m = Map(
    [
        points_interest,
        buildings
    ],
    view_state={"longitude": 4.881, "latitude": 52.359, "zoom": 14, "pitch": 45},
    basemap_style=lonboard.basemap.CartoBasemap.DarkMatterNoLabels
)
m

Explore in [Fused](https://www.fused.io/workbench/catalog/Ibis_DuckDB_Overturemaps-ca8e306b-a05a-4d48-ab40-537edd2586dc)